SealNet Detection Pipeline-- deployment
======

SealNet deployment code. This jupyter notebook allows you to make predictions on your own images with the seal detection pipeline. *WARNING* : CNNs used here were trained on panchromatic WorldView-3 imagery so results may not be satisfactory on different imagery / color bands. 

### Load packages

In [2]:
import os

### Call detection pipeline 

The following cell will tile all rasters on *inp_fldr* and generate output shapefiles with s predicted seal points for all architectures listed on *mod_arches*. For a list of supported architectures, training sets , and hyperparameter sets, see: "*utils/model_library.py*" under the main repository.

In [35]:
# select training set, model and hyperparameters
#mod_arches = ['Unet', 'UnetCntWRN', 'UnetCntWRNOccDense']
mod_arches = ['UnetCntWRNOccDense']
t_dir = 'training_set_vanilla'
hyp_set = 'E'

# setup input folder ** change here **
inp_fldr = '/home/bento/Downloads/test_scenes'

# loop through architectures
for mod_arch in mod_arches:
    out_fldr = 'test_predictions_{}'.format(mod_arch)

    # loop through images
    inp_imgs = [img for img in os.listdir('{}'.format(inp_fldr)) if img[-4:] == '.tif' and '_P004_' not in img]
    for img in inp_imgs:
        img_path = '{}/{}'.format(inp_fldr, img)
        out_fldr_img = '{}/{}'.format(out_fldr, img)
        # call pipeline 
        !python predict_raster.py --input_image=$img_path \
                                  --model_architecture=$mod_arch \
                                  --hyperparameter_set=$hyp_set \
                                  --training_set=$t_dir --test_folder=$out_fldr_img \
                                  --stride=0.25 --models_folder='saved_models_all_train'


511790 tiles created in 3 minutes and 5.97 seconds
Testing complete in 0h 13m 47s
Total predicted in WV03_20170204000736_104001002678E600_17FEB04000736-P1BS-501513717060_01_P001_u08rf3031.tif:  53

313146 tiles created in 1 minutes and 37.20 seconds
Testing complete in 0h 8m 30s
Total predicted in WV03_20170202221638_104001002983B700_17FEB02221638-P1BS-501174626090_01_P002_u08rf3031.tif:  36

553203 tiles created in 3 minutes and 35.02 seconds
Testing complete in 0h 14m 59s
Total predicted in WV03_20170301144508_104001002A465700_17MAR01144508-P1BS-501556087040_01_P001_u08rf3031.tif:  57

509434 tiles created in 2 minutes and 55.07 seconds
Testing complete in 0h 15m 18s
Total predicted in WV03_20170203044432_1040010028C45900_17FEB03044432-P1BS-501172834040_01_P001_u08rf3031.tif:  812


### Testing predictions

The following cell will merge all predicted shapefiles generated above and check them against a reference shapefile to get precision and recall. For a template on reference shapefiles see "*shape_files/seal-points-consensus.shp*" under the main repository. 

In [36]:
# setup output folder ** change here **
#mod_arches = ['Unet', 'UnetCntWRN', 'UnetCntWRNOccDense']
mod_arches = ['UnetCntWRNOccDense']
for mod_arch in mod_arches:
    inp_fldr = 'test_predictions_{}'.format(mod_arch)
    out_fldr = '{}/merged_shapefiles'.format(inp_fldr)

    # merge shapefiles
    !python merge_shapefiles.py --input_dir=$inp_fldr --output_dir=$out_fldr

    # test predictions
    shp_prediction = '{}/merged.dbf'.format(out_fldr)
    shp_ground_truth = 'shape_files/test_set_consensus/seal-points-test-consensus.dbf'
    output_file = '{}/{}_'.format(inp_fldr, mod_arch)

    !python test_sealnet.py --shp_prediction=$shp_prediction \
                            --shp_ground_truth=$shp_ground_truth \
                            --tolerance_seal=5 \
                            --tolerance_haul=20 \
                            --output_file=$output_file